# Анализ субдоменов wb.ru из Certificate Transparency Logs (crt.sh)

## Цель
Проанализировать результаты пассивной разведки субдоменов домена wb.ru, полученные из crt.sh


In [ ]:
import json
import re
from collections import defaultdict, Counter
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Настройка для отображения графиков
%matplotlib inline
try:
    plt.style.use('seaborn-v0_8-darkgrid')
except:
    plt.style.use('seaborn-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)


In [ ]:
# Загрузка данных из crt.sh
# Файл содержит JSON строку, где кавычки экранированы как "" (двойные кавычки)
import ast

with open('crt.sh.json', 'r', encoding='utf-8') as f:
    content = f.read().strip()
    
    # Файл начинается и заканчивается кавычкой - это JSON строка
    if content.startswith('"') and content.endswith('"'):
        # Используем ast.literal_eval для безопасного парсинга Python строки
        # Это правильно обработает экранированные кавычки ""
        try:
            json_string = ast.literal_eval(content)
            # Теперь парсим JSON массив
            data = json.loads(json_string)
        except Exception as e:
            print(f"Ошибка при парсинге: {e}")
            print("Пробуем альтернативный метод...")
            # Альтернативный метод: убираем внешние кавычки и заменяем "" на \"
            inner = content[1:-1]
            # Заменяем "" на \" (экранированная кавычка в JSON)
            inner = inner.replace('""', '\\"')
            data = json.loads(inner)
    else:
        # Если не строка, парсим напрямую
        data = json.loads(content)

print(f"Загружено записей: {len(data)}")
print(f"\nПример записи:")
print(json.dumps(data[0], indent=2, ensure_ascii=False))


In [ ]:
# Функция для извлечения субдоменов из name_value
def extract_subdomains(name_value, main_domain="wb.ru"):
    """
    Извлекает все субдомены из поля name_value
    name_value может содержать несколько доменов через \n или запятую
    """
    subdomains = set()
    
    # Разделяем по \n и запятым
    names = re.split(r'[\n,]+', name_value)
    
    for name in names:
        name = name.strip()
        if not name:
            continue
            
        # Убираем wildcard префикс
        if name.startswith('*.'):
            name = name[2:]
        
        # Проверяем что это субдомен wb.ru
        if name.endswith(f'.{main_domain}') or name == main_domain:
            subdomains.add(name.lower())
    
    return subdomains

# Функция для фильтрации валидных субдоменов
def is_valid_subdomain(subdomain, main_domain="wb.ru"):
    """
    Проверяет, является ли субдомен валидным
    Исключает субдомены внешних сервисов (ads, google, yandex и т.д.)
    """
    if not subdomain.endswith(f'.{main_domain}') and subdomain != main_domain:
        return False
    
    # Исключаем подозрительные паттерны
    suspicious = ['ads', 'banner', 'click', 'track', 'tracking', 
                  'google', 'yandex', 'mailru', 'facebook', 'twitter']
    
    # Проверяем части домена
    parts = subdomain.split('.')
    wb_index = -1
    for i, part in enumerate(parts):
        if part == 'wb':
            wb_index = i
            break
    
    if wb_index == -1:
        return False
    
    # Проверяем части перед wb.ru
    prefix_parts = parts[:wb_index]
    
    # Исключаем если есть подозрительные части
    for part in prefix_parts:
        if part in suspicious:
            return False
    
    # Максимум 2 уровня для обычных субдоменов (допускаем dev среды)
    if len(prefix_parts) > 2:
        # Разрешаем только для известных паттернов dev сред
        if not any(part in ['dev', 'test', 'staging', 'api'] for part in prefix_parts):
            return False
    
    return True


In [ ]:
# Извлечение всех субдоменов
all_subdomains = set()
subdomains_by_cert = []
issuers = Counter()
cert_types = Counter()

for cert in data:
    # Извлекаем субдомены
    name_value = cert.get('name_value', '')
    subdomains = extract_subdomains(name_value)
    
    # Собираем статистику
    issuer = cert.get('issuer_name', 'Unknown')
    issuers[issuer] += 1
    
    common_name = cert.get('common_name', '')
    if '*' in common_name:
        cert_types['Wildcard'] += 1
    else:
        cert_types['Single'] += 1
    
    # Сохраняем информацию о сертификате
    for subdomain in subdomains:
        if is_valid_subdomain(subdomain):
            all_subdomains.add(subdomain)
            subdomains_by_cert.append({
                'subdomain': subdomain,
                'common_name': common_name,
                'issuer': issuer,
                'not_before': cert.get('not_before', ''),
                'not_after': cert.get('not_after', ''),
                'is_wildcard': '*' in common_name
            })

print(f"Всего уникальных валидных субдоменов: {len(all_subdomains)}")
print(f"\nТоп 10 субдоменов:")
for i, subdomain in enumerate(sorted(all_subdomains)[:10], 1):
    print(f"{i}. {subdomain}")


In [ ]:
# Создание DataFrame для анализа
df = pd.DataFrame(subdomains_by_cert)

# Группировка субдоменов по категориям
def categorize_subdomain(subdomain):
    """Категоризирует субдомен по его назначению"""
    subdomain_lower = subdomain.lower()
    
    if 'api' in subdomain_lower:
        return 'API'
    elif any(x in subdomain_lower for x in ['dev', 'test', 'staging']):
        return 'Development'
    elif any(x in subdomain_lower for x in ['mail', 'smtp', 'imap', 'mx']):
        return 'Mail'
    elif any(x in subdomain_lower for x in ['cdn', 'static', 'assets', 'media']):
        return 'CDN/Static'
    elif any(x in subdomain_lower for x in ['admin', 'dashboard', 'panel']):
        return 'Admin'
    elif any(x in subdomain_lower for x in ['bank', 'finance', 'payment', 'card']):
        return 'Financial'
    elif any(x in subdomain_lower for x in ['vpn', 'fw', 'firewall']):
        return 'Security'
    elif 'www' in subdomain_lower:
        return 'WWW'
    else:
        return 'Other'

df['category'] = df['subdomain'].apply(categorize_subdomain)

print("Распределение субдоменов по категориям:")
print(df['category'].value_counts())


In [ ]:
# Визуализация: распределение по категориям
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# График 1: Распределение по категориям
category_counts = df['category'].value_counts()
axes[0].barh(category_counts.index, category_counts.values)
axes[0].set_xlabel('Количество субдоменов')
axes[0].set_title('Распределение субдоменов по категориям')
axes[0].invert_yaxis()

# График 2: Типы сертификатов
cert_type_counts = df['is_wildcard'].value_counts()
axes[1].pie(cert_type_counts.values, 
            labels=['Single Domain', 'Wildcard'],
            autopct='%1.1f%%',
            startangle=90)
axes[1].set_title('Распределение типов сертификатов')

plt.tight_layout()
plt.show()


In [ ]:
# Анализ Certificate Authorities (издателей сертификатов)
print("Топ 10 Certificate Authorities:")
print(issuers.most_common(10))

# Визуализация
plt.figure(figsize=(12, 6))
top_issuers = dict(issuers.most_common(10))
plt.barh(range(len(top_issuers)), list(top_issuers.values()))
plt.yticks(range(len(top_issuers)), list(top_issuers.keys()))
plt.xlabel('Количество сертификатов')
plt.title('Топ 10 Certificate Authorities')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


In [ ]:
# Группировка субдоменов по категориям для отчета
category_groups = defaultdict(list)

for subdomain in sorted(all_subdomains):
    category = categorize_subdomain(subdomain)
    category_groups[category].append(subdomain)

print("=" * 60)
print("ГРУППИРОВКА СУБДОМЕНОВ ПО КАТЕГОРИЯМ")
print("=" * 60)

for category in sorted(category_groups.keys()):
    subdomains = category_groups[category]
    print(f"\n{category} ({len(subdomains)} субдоменов):")
    for subdomain in sorted(subdomains):
        print(f"  - {subdomain}")


In [ ]:
# Экспорт результатов в CSV
unique_subdomains_df = pd.DataFrame({
    'subdomain': sorted(all_subdomains),
    'category': [categorize_subdomain(s) for s in sorted(all_subdomains)]
})

unique_subdomains_df.to_csv('wb_ru_subdomains_unique.csv', index=False, encoding='utf-8')
print(f"Экспортировано {len(unique_subdomains_df)} уникальных субдоменов в wb_ru_subdomains_unique.csv")

# Экспорт детальной информации
df.to_csv('wb_ru_subdomains_detailed.csv', index=False, encoding='utf-8')
print(f"Экспортировано {len(df)} записей в wb_ru_subdomains_detailed.csv")


In [ ]:
# Статистика по wildcard сертификатам
wildcard_certs = df[df['is_wildcard'] == True]
print(f"Сертификатов с wildcard: {len(wildcard_certs)}")
print(f"Уникальных wildcard субдоменов: {wildcard_certs['subdomain'].nunique()}")

print("\nПримеры wildcard сертификатов:")
for idx, row in wildcard_certs.head(10).iterrows():
    print(f"  {row['common_name']} -> {row['subdomain']}")


In [ ]:
# Итоговая статистика
print("=" * 60)
print("ИТОГОВАЯ СТАТИСТИКА")
print("=" * 60)
print(f"Всего записей в crt.sh: {len(data)}")
print(f"Уникальных валидных субдоменов: {len(all_subdomains)}")
print(f"Уникальных Certificate Authorities: {len(issuers)}")
print(f"\nРаспределение по категориям:")
for category, count in df['category'].value_counts().items():
    print(f"  {category}: {count}")
print(f"\nТипы сертификатов:")
print(f"  Wildcard: {cert_types['Wildcard']}")
print(f"  Single Domain: {cert_types['Single']}")
